Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуем корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## 1. Загрузка данных

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import r2_score

In [2]:
clients = pd.read_csv('/datasets/insurance.csv')
clients.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


## 2. Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** Качество линейной регрессии не изменится

**Обоснование:** если $ w = (X^T X)^{-1} X^T y $, и $ a = Xw $, то $ a = X (X^T X)^{-1} X^T y $

Если мы умножим X на некую обратимую (следовательно квадратную) матрицу Z, то новый вектор предсказаний b будет выглядеть так:

$ b = (XZ) ((XZ)^T (XZ))^{-1} (XZ)^T y = XZ (Z^T X^T X Z)^{-1} Z^T X^T  y = XZ (X^T X Z)^{-1} (Z^T)^{-1} Z^T X^T y = XZ Z^{-1} (X^T X)^{-1} E X^T y = X E (X^T X)^{-1} X^T y = X (X^T X)^{-1} X^T y = a        $

Соответственно, если старый и новый векторы предсказаний одинаковы, то и метрики качества будут иметь одно значение.

## 3. Алгоритм преобразования

**Алгоритм**

1. Узнать модель на исходных признаках и подсчитать значение метрики качества.

2. Умножить матрицу значений признаков размера (m, n) на случайную обратимую матрицу размера (n, n) и получить матрицу новых значений признаков.

3. Переобучить модель на новых признаках и узнать новое значение метрики качества.

4. Убедиться, что метрика качества осталась прежней.

**Обоснование**

В результате преобразования мы получим новые значения матрицы признаков, которые в случае перехвата не дадут злоумышленнику восстановить исходные данные. Теоретическое обоснование того, что качество модели линейной регрессии останется прежним, приведено в пункте 2.

## 4. Проверка алгоритма

In [4]:
features = clients.drop('Страховые выплаты', axis=1)
target = clients['Страховые выплаты']
features.shape

(5000, 4)

Узнаем метрику r2 на исходных данных

In [5]:
model = LinearRegression()
model.fit(features, target)
predictions = model.predict(features)
print(r2_score(target, predictions))

0.42494550286668


Преобразуем признаки, умножив их на случайную обратимую матрицу нужного размера

In [6]:
array_features = np.array(features)
randmatrix = np.random.rand(features.shape[1], features.shape[1])
new = array_features @ randmatrix
new

array([[37564.60738615, 31190.37613304, 40770.71671106, 43054.27776317],
       [28783.51092312, 23898.21428391, 31244.54172357, 32984.78021678],
       [15907.68235092, 13207.03244268, 17268.4201099 , 18227.9559664 ],
       ...,
       [25671.59430711, 21316.91867886, 27861.56324284, 29426.73435419],
       [24764.45102805, 20564.59496529, 26878.13808558, 28386.97563383],
       [30746.85148369, 25530.11407311, 33369.6169257 , 35242.29192061]])

Проверим, обратима ли матрица

In [7]:
np.linalg.inv(randmatrix)

array([[ 1.77608317,  1.80907689, -0.29361161, -1.47828024],
       [ 3.53086775,  3.55538449, -4.0412358 ,  0.34744239],
       [-1.94312582, -0.30518918,  1.32992131,  0.6435689 ],
       [-2.26744781, -3.86412848,  3.07647071,  0.42867733]])

Переобучим модель на новых данных и сравним новое значение r2 с исходным

In [8]:
model2 = LinearRegression()
model2.fit(new, target)
predictions2 = model2.predict(new)
print(r2_score(target, predictions))

0.42494550286668


Как мы видим, качество модели линейной регрессии осталось прежним.

### Вывод

Мы доказали, что умножение матрицы значений признаков на обратимую матрицу не ведёт к изменению качества работы модели линейной регрессии. 

Опираясь на это положение, мы преобразовали данные клиентов, умножив их на случайную обратимую матрицу, в результате чего восстановить исходные данные, не обладая нужной матрицей, будет сложно. Вместе с тем точность модели линейной регрессии на новых данных осталась такой же.